## Setup

In [1]:
using RigidBodyDynamics
using StaticArrays
using SymPy

## Create symbolic parameters
* Masses: $m_1, m_2$
* Mass moments of inertia (about center of mass): $I_1, I_2$
* Link lengths: $l_1, l_2$
* Center of mass locations (w.r.t. preceding joint axis): $c_1, c_2$
* Gravitational acceleration: $g$

In [2]:
inertias = @syms m_1 m_2 I_1 I_2 positive = true
lengths = @syms l_1 l_2 c_1 c_2 real = true
gravitationalAcceleration = @syms g real = true
params = [inertias..., lengths..., gravitationalAcceleration...]

9-element Array{SymPy.Sym,1}
[m_1]
[   ]
[m_2]
[   ]
[I_1]
[   ]
[I_2]
[   ]
[l_1]
[   ]
[l_2]
[   ]
[c_1]
[   ]
[c_2]
[   ]
[ g ]

## Create double pendulum `Mechanism`

A `Mechanism` contains the joint layout and inertia parameters, but no state information.

In [3]:
T = Sym # the 'scalar type' of the Mechanism we'll construct
axis = SVector(zero(T), one(T), zero(T)) # axis of rotation for each of the joints
doublePendulum = Mechanism(RigidBody{T}("world"); gravity = SVector(zero(T), zero(T), g))
world = root_body(doublePendulum) # the fixed 'world' rigid body

# Attach the first (upper) link to the world via a revolute joint named 'shoulder'
inertia1 = SpatialInertia(CartesianFrame3D("upper_link"), I_1 * axis * axis.', m_1 * SVector(zero(T), zero(T), c_1), m_1)
body1 = RigidBody(inertia1)
joint1 = Joint("shoulder", Revolute(axis))
joint1ToWorld = eye(RigidBodyDynamics.Transform3DS{T}, frame_before(joint1), default_frame(world))
attach!(doublePendulum, world, joint1, joint1ToWorld, body1)

# Attach the second (lower) link to the world via a revolute joint named 'elbow'
inertia2 = SpatialInertia(CartesianFrame3D("lower_link"), I_2 * axis * axis.', m_2 * SVector(zero(T), zero(T), c_2), m_2)
body2 = RigidBody(inertia2)
joint2 = Joint("elbow", Revolute(axis))
joint2ToBody1 = Transform3D(frame_before(joint2), default_frame(body1), SVector(zero(T), zero(T), l_1))
attach!(doublePendulum, body1, joint2, joint2ToBody1, body2)

Spanning tree:
Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
    Vertex: lower_link, Edge: elbow
No non-tree joints.

## Create `MechanismState` associated with the double pendulum `Mechanism`

A `MechanismState` stores all state-dependent information associated with a `Mechanism`.

In [4]:
x = MechanismState{T}(doublePendulum);

In [5]:
# Set the joint configuration vector of the MechanismState to a new vector of symbolic variables
configuration(x)[:] = [symbols("q_$i", real = true) for i = 1 : num_positions(x)]

2-element Array{SymPy.Sym,1}
[q_1]
[   ]
[q_2]

In [6]:
# Set the joint velocity vector of the MechanismState to a new vector of symbolic variables
velocity(x)[:] = [symbols("v_$i", real = true) for i = 1 : num_positions(x)]

2-element Array{SymPy.Sym,1}
[v_1]
[   ]
[v_2]

## Compute dynamical quantities in symbolic form

In [7]:
# Mass matrix
M = mass_matrix(x)
map!(simplify, M.data, M.data) # Note: M is a Symmetric matrix type; need to simplify the underlying data
full(M) # convert to full form so that it is pretty-printed (minor bug in SymPy.jl)

2×2 Array{SymPy.Sym,2}
[                                    2                            ]
[I_1 + I_2 + 2*c_2*l_1*cos(q_2) + l_1 *m_2  I_2 + c_2*l_1*cos(q_2)]
[                                                                 ]
[         I_2 + c_2*l_1*cos(q_2)                     I_2          ]

In [8]:
# Kinetic energy
simplify(kinetic_energy(x))

       2          2                        2                                  
I_1*v_1    I_2*v_1                  I_2*v_2               2                   
-------- + -------- + I_2*v_1*v_2 + -------- + c_2*l_1*v_1 *cos(q_2) + c_2*l_1
   2          2                        2                                      

                       2        2
                    l_1 *m_2*v_1 
*v_1*v_2*cos(q_2) + -------------
                          2      

In [9]:
# Potential energy
simplify(gravitational_potential_energy(x))

-g*(c_1*cos(q_1) + c_2*cos(q_1 + q_2) + l_1*m_2*cos(q_1))